# YOLOv8 Fine-Tuning para Detección de EPP
## Safety Vision AI - Hard Hat Detection

**Dataset:** Hard Hat Detection (5,269 training images)
**Clases:** head, helmet, person
**Fecha:** 2026-01-28

## 1️⃣ Imports y Verificaciones

In [ ]:
from ultralytics import YOLO
import torch
import os
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"✅ CUDA version: {torch.version.cuda}")
else:
    print("⚠️  Running on CPU (esto será más lento pero funcional)")

## 2️⃣ Verificar Dataset

In [ ]:
# Verificar dataset
dataset_path = Path("../datasets/helmet_vest_detection")
data_yaml = dataset_path / "data.yaml"

if not data_yaml.exists():
    print("❌ ERROR: No se encontró data.yaml")
    print(f"Verifica que el dataset esté en: {dataset_path}")
else:
    print(f"✅ Dataset encontrado: {data_yaml}")
    
    # Contar imágenes
    train_images = list((dataset_path / "train" / "images").glob("*.jpg")) + \
                   list((dataset_path / "train" / "images").glob("*.png"))
    test_images = list((dataset_path / "test" / "images").glob("*.jpg")) + \
                   list((dataset_path / "test" / "images").glob("*.png"))
    
    print(f"\n📊 ESTADÍSTICAS DEL DATASET:")
    print(f"   🔹 Imágenes de entrenamiento: {len(train_images):,}")
    print(f"   🔹 Imágenes de validación: {len(test_images):,}")
    print(f"   🔹 Total: {len(train_images) + len(test_images):,}")
    
    # Verificar labels
    train_labels = list((dataset_path / "train" / "labels").glob("*.txt"))
    print(f"\n📝 Labels de entrenamiento: {len(train_labels):,}")
    
    if len(train_images) > 0:
        print(f"\n✅ Dataset listo para entrenar!")
    else:
        print("\n❌ No se encontraron imágenes")

## 3️⃣ Visualizar Muestras del Dataset

In [ ]:
# Visualizar algunas imágenes de ejemplo
import random

if len(train_images) > 0:
    # Seleccionar 6 imágenes aleatorias
    sample_images = random.sample(train_images, min(6, len(train_images)))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(sample_images):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].set_title(f"Imagen {idx+1}\n{img_path.name}")
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Muestras del Dataset de Entrenamiento', fontsize=16, y=1.02)
    plt.show()
else:
    print("No hay imágenes para visualizar")

## 4️⃣ Cargar Modelo Base YOLOv8

In [ ]:
# Cargar modelo base pre-entrenado
# Opciones: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium)
# Recomendado: yolov8n.pt para empezar (más rápido)

model = YOLO('yolov8n.pt')
print("✅ Modelo base YOLOv8n cargado correctamente")
print(f"📊 Parámetros del modelo: {sum(p.numel() for p in model.model.parameters()):,}")

## 5️⃣ Configuración de Entrenamiento

In [ ]:
# Configuración de hiperparámetros
CONFIG = {
    'data': str(data_yaml),
    'epochs': 50,              # Número de epochs (min 30, ideal 50-100)
    'imgsz': 640,              # Tamaño de imagen
    'batch': 16,               # Batch size (reducir si hay error de memoria: 8, 4, 2)
    'device': 0,               # 0 = GPU, 'cpu' = CPU
    'project': '../models_assets',
    'name': 'yolov8_helmet_detection',
    'patience': 10,            # Early stopping
    'save': True,
    'plots': True,
    'workers': 8,              # Número de workers para cargar datos
    'cache': False,            # Cache de imágenes (True si tienes RAM suficiente)
    
    # Data Augmentation
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10.0,           # Rotación
    'translate': 0.1,          # Traslación
    'scale': 0.5,              # Escalado
    'flipud': 0.0,             # Flip vertical
    'fliplr': 0.5,             # Flip horizontal
}

print("⚙️  CONFIGURACIÓN DE ENTRENAMIENTO:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")

## 6️⃣ ENTRENAR EL MODELO 🚀

⏰ **Tiempo estimado:**
- Con GPU: 40-80 minutos (50 epochs)
- Con CPU: 4-8 horas (no recomendado)

**Nota:** Puedes detener en cualquier momento con el botón Stop. El mejor modelo se guarda automáticamente.

In [ ]:
# ENTRENAR
print("\n" + "="*60)
print("🚀 INICIANDO ENTRENAMIENTO...")
print("="*60 + "\n")

results = model.train(**CONFIG)

print("\n" + "="*60)
print("🎉 ¡ENTRENAMIENTO COMPLETADO!")
print("="*60)

## 7️⃣ Evaluar el Modelo

In [ ]:
# Evaluar en el conjunto de validación
print("\n📊 EVALUANDO MODELO...\n")
metrics = model.val()

print("\n" + "="*60)
print("📈 MÉTRICAS DEL MODELO:")
print("="*60)
print(f"\n🎯 mAP@0.5:        {metrics.box.map50:.4f}")
print(f"🎯 mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"🎯 Precision:      {metrics.box.mp:.4f}")
print(f"🎯 Recall:         {metrics.box.mr:.4f}")

# Evaluación por clase
print("\n📊 MÉTRICAS POR CLASE:")
class_names = ['head', 'helmet', 'person']
for i, name in enumerate(class_names):
    if hasattr(metrics.box, 'maps'):
        print(f"   {name}: mAP@0.5 = {metrics.box.maps[i]:.4f}")

## 8️⃣ Guardar el Mejor Modelo

In [ ]:
import shutil

# Ubicación del mejor modelo
best_model_path = Path("../models_assets/yolov8_helmet_detection/weights/best.pt")

if best_model_path.exists():
    # Copiar a una ubicación más accesible
    destination = Path("../models_assets/yolov8_helmet_vest_best.pt")
    shutil.copy(best_model_path, destination)
    
    print("\n" + "="*60)
    print("💾 MODELO GUARDADO")
    print("="*60)
    print(f"\n✅ Mejor modelo guardado en: {destination}")
    print(f"📁 Tamaño del archivo: {destination.stat().st_size / (1024*1024):.2f} MB")
    print(f"\n📂 Archivos del entrenamiento en:")
    print(f"   {best_model_path.parent.parent}")
else:
    print("\n❌ No se encontró el modelo entrenado")
    print(f"Buscado en: {best_model_path}")

## 9️⃣ Visualizar Curvas de Entrenamiento

In [ ]:
# Mostrar gráficas de entrenamiento
results_path = Path("../models_assets/yolov8_helmet_detection")

if results_path.exists():
    # Buscar imagen de resultados
    results_img = results_path / "results.png"
    
    if results_img.exists():
        print("\n📊 CURVAS DE ENTRENAMIENTO:\n")
        img = Image.open(results_img)
        plt.figure(figsize=(16, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Métricas de Entrenamiento', fontsize=16)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️  Gráfica de resultados no encontrada")
else:
    print("⚠️  Carpeta de resultados no encontrada")

## 🔟 Probar el Modelo en Imágenes de Validación

In [ ]:
# Cargar el mejor modelo y probar
if destination.exists() and len(test_images) > 0:
    print("\n🎯 PROBANDO EL MODELO EN IMÁGENES DE VALIDACIÓN\n")
    
    # Cargar mejor modelo
    best_model = YOLO(str(destination))
    
    # Seleccionar 6 imágenes aleatorias del conjunto de validación
    sample_test_images = random.sample(test_images, min(6, len(test_images)))
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(sample_test_images):
        # Hacer predicción
        results = best_model(str(img_path), verbose=False)
        
        # Dibujar predicciones
        annotated = results[0].plot()
        
        # Mostrar
        axes[idx].imshow(annotated[..., ::-1])  # BGR to RGB
        axes[idx].set_title(f"Predicción {idx+1}", fontsize=12)
        axes[idx].axis('off')
        
        # Imprimir detecciones
        print(f"\nImagen {idx+1}: {img_path.name}")
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            name = results[0].names[cls]
            print(f"  ➤ {name}: {conf:.2f}")
    
    plt.tight_layout()
    plt.suptitle('Predicciones del Modelo en Imágenes de Validación', fontsize=16, y=1.02)
    plt.show()
else:
    print("⚠️  No se pudo probar el modelo")

## ✅ RESUMEN FINAL

In [ ]:
print("\n" + "="*70)
print("🎉 FASE 1 COMPLETADA - FINE-TUNING EXITOSO")
print("="*70)

if destination.exists():
    print("\n✅ MODELO ENTRENADO Y GUARDADO:")
    print(f"   📁 Ubicación: {destination}")
    print(f"   💾 Tamaño: {destination.stat().st_size / (1024*1024):.2f} MB")
    
    print("\n📊 DATASET UTILIZADO:")
    print(f"   🔹 Entrenamiento: {len(train_images):,} imágenes")
    print(f"   🔹 Validación: {len(test_images):,} imágenes")
    print(f"   🔹 Clases: head, helmet, person")
    
    print("\n🎯 PRÓXIMOS PASOS:")
    print("   1️⃣ Revisar métricas (mAP debe ser > 0.75)")
    print("   2️⃣ Commitear cambios a Git")
    print("   3️⃣ Continuar con FASE 2: Pipeline de Inferencia")
    
    print("\n💡 COMANDOS PARA GIT:")
    print("   git add .")
    print(f"   git commit -m 'feat: train YOLOv8 model with mAP={metrics.box.map50:.3f}'")
    print("   git push")
    
    print("\n🚀 ¡EXCELENTE TRABAJO!")
    print("="*70 + "\n")
else:
    print("\n⚠️  Revisa que el entrenamiento haya completado correctamente")